# Geotemporal Trends in Urbanization: Cameroon
*Using yearly estimates (2000-2015) of population, built-area, and economic indicators to track city-by-city growth and change over time.*

---

### Research questions 

#### 1. How has the size of Settlement X changed over time? 

- Population size 

- Geographical extents 

- Population density 

#### 2. In what year did Settlement X become a new urban class?  

- From semi-dense to high-density city 

- Small settlement area to built-up area 

- When a hamlet area or small settlement area first appeared

#### 3. Is there a discernable pattern between the spatio-temporal distribution of economic density and population density? 

#### 4. How much of urban space attributable to City X is outside of the administrative limits of the city? 

- When did this fragment(s) appear? 

- Which district/municipality/authority has purview over the fragment(s)? 

#### 5. For the questions above, how does the answer change based on different understandings of urban limits? 

- Scenario A: where "city" is delimited by an official administrative boundary 

- Scenario B: where "city" includes all contiguous (and near-contiguous) built up area 

#### 6. Subnational and inter-national comparisons. Examples: 

- Compare the rates (pop, build-up, economic…) of the fastest growing settlement of each ADM1 region. 

- Which African metropoles experience the most vs. the least fragmentation? Is there a confluence between amount of urban fragmentation and rate of densification? 

---

### Datasets
1. Most up-to-date administrative boundaries: **ADM3.**
2. Built-up area, yearly: **World Settlement Footprint Evolution.** Resolution: 30m. 
3. Settlement types: **GRID3 settlement extents.** Captured between 2009-2019.
4. Population, yearly: **WorldPop.** UN-adjusted, unconstrained. Resolution: 100m.
5. Nighttime lights, yearly: **Harmonization of DMSP and VIIRS.** Resolution: 1km.
6. City names: **UCDB, Africapolis, and GeoNames.**

---

### Previous steps (ArcGIS Pro and Python)

##### DATA PREP 

Projection for all datasets --> Africa Albers Equal Area Conic 

Remove unnecessary fields (e.g. extra fields in gazetteer data) 

##### WSFE AND ADM3 

RASTER MATH: "Join" ADM ID onto WSFE by creating unique string. 

VECTOR MATH: Split unique ID from raster math into separate columns, and remove non-buildup years (0000). 

##### GRID3 AND ADM3 

RASTER MATH: "Join" ADM3 ID onto GRID3 by creating unique string. 

VECTOR MATH: Split unique ID from raster math into separate columns. 

##### ADD NAMES 

Join features: UCDB, Africapolis, and GeoNames

##### WSFE AND GRID3 

NEAR JOIN: Join GRID3 ID onto WSFE by spatial (within a distance) and attribute matching. 
 
##### WSFE CUMULATIVE 

DISSOLVE BY YEAR SETS: Create separate feature layers of each cumulative year. 

---

### Associating yearly datasets. (current notebook)

#### Prepare workspace

In [2]:
# Installs

#import sys
#!{sys.executable} -m pip install voronoi-diagram-for-polygons xarray-spatial rioxarray pygeos

     --------------------------------------- 10.9/10.9 MB 46.7 MB/s eta 0:00:00
     ---------------------------------------- 52.2/52.2 kB 2.8 MB/s eta 0:00:00
     --------------------------------------- 18.2/18.2 MB 40.9 MB/s eta 0:00:00
     ------------------------------------- 943.1/943.1 kB 29.2 MB/s eta 0:00:00
     ---------------------------------------- 76.5/76.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.8/86.8 kB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 34.9 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 35.9 MB/s eta 0:00:00
     -------------------------------------- 140.8/140.8 kB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 kB 3.0 MB/s eta 0:00:00
     --------------------------------------- 18.5/18.5 MB 38.4 MB/s eta 0:00:00
     ---------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\wb527163\\.conda\\envs\\geo\\Lib\\site-packages\\~ornado\\speedups.pyd'
Consider using the `--user` option or check the permissions.



In [17]:
# import sys
# !{sys.executable} -m pip install --user --upgrade pygeos

In [79]:
# !{sys.executable} -m pip install --user ogr

     ---------------------------------------- 76.7/76.7 kB 4.2 MB/s eta 0:00:00
     ------------------------------------- 182.5/182.5 kB 10.8 MB/s eta 0:00:00
     ------------------------------------- 293.4/293.4 kB 17.7 MB/s eta 0:00:00
     -------------------------------------- 127.9/127.9 kB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 63.1/63.1 kB 3.5 MB/s eta 0:00:00
     ------------------------------------- 212.1/212.1 kB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 54.4/54.4 kB 2.9 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [99]:
import os, sys
import glob
import re
import time

import geopandas as gpd 
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, shape
from shapely.ops import voronoi_diagram
from shapely.validation import make_valid
from longsgis import voronoiDiagram4plg 
import fiona
import numpy

from xrspatial import zonal_stats 
import xarray as xr 
import rasterio 
import rioxarray 
from osgeo import gdal, osr, ogr

In [6]:
ProjectFolder = os.getcwd()
WorldPopFolder = os.path.join(ProjectFolder, "WorldPop_tifs_albers")
print(ProjectFolder)
print(WorldPopFolder)

C:\Users\wb527163\GEO_Cdrive_Grace\urban_growth
C:\Users\wb527163\GEO_Cdrive_Grace\urban_growth\WorldPop_tifs_albers


#### Input datasets

In [7]:
BuiltAreaList = fiona.listlayers("WSFE_cumulativelayers.gdb")
print(BuiltAreaList)

['cu_sel_1999', 'cu_sel_2000', 'cu_sel_2001', 'cu_sel_2002', 'cu_sel_2003', 'cu_sel_2004', 'cu_sel_2005', 'cu_sel_2006', 'cu_sel_2007', 'cu_sel_2008', 'cu_sel_2009', 'cu_sel_2010', 'cu_sel_2011', 'cu_sel_2012', 'cu_sel_2013', 'cu_sel_2014', 'cu_sel_2015']


In [4]:
Boundary = gpd.read_file('WSFE.gdb', layer=1)#; Boundary.crs = "ESRI:102022"
print(Boundary.info())
print(Boundary.crs)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID_COM        360 non-null    int64   
 1   COMMUNES      360 non-null    object  
 2   ID_DEP        360 non-null    int64   
 3   DEPARTEMEN    360 non-null    object  
 4   ID_REG        360 non-null    int64   
 5   REGIONS       360 non-null    object  
 6   Shape_Length  360 non-null    float64 
 7   Shape_Area    360 non-null    float64 
 8   geometry      360 non-null    geometry
dtypes: float64(2), geometry(1), int64(3), object(3)
memory usage: 25.4+ KB
None
PROJCS["Africa_Albers_Equal_Area_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_cente

In [18]:
PopRasterList = [rioxarray.open_rasterio(item, masked=True) 
                 for item in glob.glob(r'C:\Users\wb527163\GEO_Cdrive_Grace\urban_growth\WorldPop_tifs_albers' 
                                       + '**/*.tif', recursive=True)] 

print(PopRasterList)

for item in PopRasterList:
    print(item.rio.crs) 

#### Settlement extents by year (cumulative built areas)

In [5]:
BuiltAllYears = gpd.GeoDataFrame(
    columns=['GRID3_splitID', 'MAX_year', 'Shape_Length', 'Shape_Area', 'CuYear', 'geometry'], 
    geometry='geometry', crs = "ESRI:102022")
print(BuiltAllYears.info())
print(BuiltAllYears.crs)
print(BuiltAllYears.head())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   GRID3_splitID  0 non-null      object  
 1   MAX_year       0 non-null      object  
 2   Shape_Length   0 non-null      object  
 3   Shape_Area     0 non-null      object  
 4   CuYear         0 non-null      object  
 5   geometry       0 non-null      geometry
dtypes: geometry(1), object(5)
memory usage: 108.0+ bytes
None
ESRI:102022
Empty GeoDataFrame
Columns: [GRID3_splitID, MAX_year, Shape_Length, Shape_Area, CuYear, geometry]
Index: []


In [54]:
for item in BuiltAreaList:
    CuYear = re.sub(r'[^0-9]', '', item)
    TempItem = gpd.read_file("WSFE_cumulativelayers.gdb", layer=item)
    TempItem["CuYear"] = CuYear
    BuiltAllYears = pd.concat([BuiltAllYears, TempItem])
    
BuiltAllYears.sample(20)

,GRID3_splitID,MAX_year,Shape_Length,Shape_Area,CuYear,geometry
5580,44453.0,2011,395.4604,7181.121822,2014,"MULTIPOLYGON (((-1590070.161 681154.342, -1590..."
10125,192661.0,2006,1864.3124,73406.949158,2006,"MULTIPOLYGON (((-1609193.487 672425.970, -1609..."
6597,57144.0,2015,677.9319,21543.358405,2015,"MULTIPOLYGON (((-1427253.542 538364.958, -1427..."
1374,4827.0,2011,1016.8976,32713.977756,2012,"MULTIPOLYGON (((-1648993.733 470826.003, -1648..."
1820,9985.0,1985,112.9887,797.902895,2006,"MULTIPOLYGON (((-1457223.777 366170.282, -1457..."
330,360.0,2006,8813.1140,382992.963299,2006,"MULTIPOLYGON (((-1317937.040 901510.422, -1317..."
7533,100766.0,2000,564.9430,10372.727755,2014,"MULTIPOLYGON (((-1427564.260 704712.472, -1427..."
2973,16173.0,2006,903.9097,20745.459747,2011,"MULTIPOLYGON (((-1498831.840 468763.960, -1498..."
3699,23191.0,1985,1581.8411,62236.349580,2006,"MULTIPOLYGON (((-1344771.840 395519.079, -1344..."
10072,191702.0,2011,2655.2319,41490.864411,2013,"MULTIPOLYGON (((-1191474.513 513140.246, -1191..."


In [55]:
print(BuiltAllYears.crs)

ESRI:102022


#### Thiessen polygons (Voronoi polygons): For each year, demarcate the surrounding space which is closest to a particular feature than to any other feature in the year set.
#### Then, buffer area of each built-up polygon and use that buffer to clip the Thiessen areas.

In [ ]:
#If CRSs do not match:
#PopRasterList = [item.rio.reproject_match(BuiltArea) for item in PopRasterList]
#Boundary = Boundary.to_crs(BuiltArea.crs)

In [5]:
Boundary = Boundary.dissolve()
Boundary.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   geometry      1 non-null      geometry
 1   ID_COM        1 non-null      int64   
 2   COMMUNES      1 non-null      object  
 3   ID_DEP        1 non-null      int64   
 4   DEPARTEMEN    1 non-null      object  
 5   ID_REG        1 non-null      int64   
 6   REGIONS       1 non-null      object  
 7   Shape_Length  1 non-null      float64 
 8   Shape_Area    1 non-null      float64 
dtypes: float64(2), geometry(1), int64(3), object(3)
memory usage: 188.0+ bytes


In [6]:
# BuiltThiessenBuffer = gpd.GeoDataFrame(
#     columns=['GRID3_splitID', 'MAX_year', 'Shape_Length', 'Shape_Area', 'CuYear', 'geometry'], 
#     geometry='geometry', crs = "ESRI:102022")

In [8]:
for item in BuiltAreaList:
    print('Loading layer %s. %s\n' % (item, time.ctime()))
    CuYear = re.sub(r'[^0-9]', '', item) # Pull the year of feature layer (e.g. "2005") from the numeric portion of the layer name.
    Layer = gpd.read_file("WSFE_cumulativelayers.gdb", layer=item) # Read in the layer as a geodataframe.
    print('Create a buffer around the features in the original layer. %s\n' % time.ctime())
    BufferLayer = Layer
    BufferLayer['geometry'] = BufferLayer['geometry'].apply(make_valid) # This is a workaround for any null geometries.
    BufferLayer['geometry'] = BufferLayer['geometry'].buffer(2000) # Create a 2km buffer around the original feature.
    BufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['Buff_', CuYear, '.shp']))
    print('Buffered version finished and saved to file. %s\n' % time.ctime())
    del CuYear, Layer, BufferLayer

Loading layer cu_sel_1999. Fri Oct  7 08:24:09 2022

Create a buffer around the features in the original layer. Fri Oct  7 08:24:13 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_1816\49490589.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  BufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['Buff_', CuYear, '.shp']))


Buffered version finished and saved to file. Fri Oct  7 10:02:13 2022

Loading layer cu_sel_2000. Fri Oct  7 10:02:13 2022

Create a buffer around the features in the original layer. Fri Oct  7 10:02:17 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_1816\49490589.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  BufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['Buff_', CuYear, '.shp']))


Buffered version finished and saved to file. Fri Oct  7 11:31:10 2022

Loading layer cu_sel_2001. Fri Oct  7 11:31:10 2022

Create a buffer around the features in the original layer. Fri Oct  7 11:31:14 2022



...

Buffered version finished and saved to file. Sat Oct  8 00:56:05 2022

Loading layer cu_sel_2014. Sat Oct  8 00:56:05 2022

Create a buffer around the features in the original layer. Sat Oct  8 00:56:09 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_1816\49490589.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  BufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['Buff_', CuYear, '.shp']))


Buffered version finished and saved to file. Sat Oct  8 01:44:28 2022

Loading layer cu_sel_2015. Sat Oct  8 01:44:29 2022

Create a buffer around the features in the original layer. Sat Oct  8 01:44:32 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_1816\49490589.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  BufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['Buff_', CuYear, '.shp']))


Buffered version finished and saved to file. Sat Oct  8 02:30:12 2022



In [14]:
for item in ['cu_sel_2011', 'cu_sel_2012', 'cu_sel_2013', 'cu_sel_2014', 'cu_sel_2015']:
# Error happened in cu_sel_2010. See OneNote page for details.
# ValueError: No Shapely geometry can be created from null value
# for item in BuiltAreaList:
    print('Loading layer %s. %s\n' % (item, time.ctime()))
    CuYear = re.sub(r'[^0-9]', '', item) # Pull the year of feature layer (e.g. "2005") from the numeric portion of the layer name.
    Layer = gpd.read_file("WSFE_cumulativelayers.gdb", layer=item) # Read in the layer as a geodataframe.
    Buffer = gpd.read_file(filename=''.join(['Buff_', CuYear, '.shp']))
    print('Loaded. Assigning year field. %s\n' % time.ctime())
    Layer["CuYear"] = CuYear # Give geodataframe a field where every value is the year of cumulative buildup represented by the layer. This will be useful if concatenating all the layers together into a single dataset.
    print('Assigned. Drawing Thiessen (Voronoi) polygons using buffer as the bounding area. %s\n' % time.ctime())
    ThiessenLayer = voronoiDiagram4plg(Layer, Buffer) # Demarcate the area around each feature which is closer to that feature than any other feature.
    ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThBuf_', CuYear, '.shp']))
    print('Polygons drawn and written to file. %s\n' % time.ctime())
    del CuYear, Layer, ThiessenLayer

Loading layer cu_sel_2011. Mon Oct 10 10:14:46 2022

Loaded. Assigning year field. Mon Oct 10 10:14:51 2022

Assigned. Drawing Thiessen (Voronoi) polygons using buffer as the bounding area. Mon Oct 10 10:14:51 2022



C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:32: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gs = gpd.GeoSeries([smp_vd]).explode()
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\l

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\w

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\w

Polygons drawn and written to file. Mon Oct 10 12:11:29 2022

Loading layer cu_sel_2012. Mon Oct 10 12:11:29 2022

Loaded. Assigning year field. Mon Oct 10 12:11:34 2022

Assigned. Drawing Thiessen (Voronoi) polygons using buffer as the bounding area. Mon Oct 10 12:11:34 2022



...

Polygons drawn and written to file. Mon Oct 10 16:45:45 2022

Loading layer cu_sel_2015. Mon Oct 10 16:45:46 2022

Loaded. Assigning year field. Mon Oct 10 16:45:51 2022

Assigned. Drawing Thiessen (Voronoi) polygons using buffer as the bounding area. Mon Oct 10 16:45:51 2022



C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:32: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gs = gpd.GeoSeries([smp_vd]).explode()
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\l

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\w

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:56: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return MultiPolygon(Polygon(p.exterior) for p in plg)
C:\Users\w

Polygons drawn and written to file. Mon Oct 10 18:19:00 2022



In [42]:
# This was the only year that gave me trouble. Running it outside of the list now that the issues are resolved.
item = 'cu_sel_2010'
print('Loading layer %s. %s\n' % (item, time.ctime()))
CuYear = re.sub(r'[^0-9]', '', item) # Pull the year of feature layer (e.g. "2005") from the numeric portion of the layer name.
Layer = gpd.read_file("WSFE_cumulativelayers.gdb", layer=item) # Read in the layer as a geodataframe.
Buffer = gpd.read_file(filename=''.join(['Buff_', CuYear, '.shp']))
print('Loaded. Assigning year field. %s\n' % time.ctime())
Layer["CuYear"] = CuYear # Give geodataframe a field where every value is the year of cumulative buildup represented by the layer. This will be useful if concatenating all the layers together into a single dataset.
print('Assigned. Drawing Thiessen (Voronoi) polygons. %s\n' % time.ctime())
Layer['geometry'] = Layer['geometry'].apply(make_valid).buffer(0.1).simplify(10) # Using several workarounds for any null geometries. Our inputs don't need a high level of precision.
Buffer['geometry'] = Buffer['geometry'].apply(make_valid)
ThiessenLayer = voronoiDiagram4plg(Layer, Buffer) # Demarcate the area around each feature which is closer to that feature than any other feature.
ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThBuf_', CuYear, '.shp']))
print('Polygons drawn and written to file. %s\n' % time.ctime())

Loading layer cu_sel_2010. Thu Oct 13 17:28:00 2022

Loaded. Assigning year field. Thu Oct 13 17:28:06 2022

Assigned. Drawing Thiessen (Voronoi) polygons. Thu Oct 13 17:28:06 2022



C:\Users\wb527163\.conda\envs\geo\lib\site-packages\longsgis\longsgis.py:32: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gs = gpd.GeoSeries([smp_vd]).explode()
C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
TopologyException: side location conflict at -1492504.3774815281 714118.76582411525. This can occur if the input geometry is invalid.


ValueError: No Shapely geometry can be created from null value

In [51]:
BuiltListless2010 = BuiltAreaList.remove('cu_sel_2010')
BuiltAreaList

['cu_sel_1999',
 'cu_sel_2000',
 'cu_sel_2001',
 'cu_sel_2002',
 'cu_sel_2003',
 'cu_sel_2004',
 'cu_sel_2005',
 'cu_sel_2006',
 'cu_sel_2007',
 'cu_sel_2008',
 'cu_sel_2009',
 'cu_sel_2011',
 'cu_sel_2012',
 'cu_sel_2013',
 'cu_sel_2014',
 'cu_sel_2015']

In [58]:
print('Thiessen polygons did not retain feature attributes. Need to join back on spatially. \n\n')
for item in BuiltAreaList:
    print('Year: %s %s\n' % (item, time.ctime()))
    CuYear = re.sub(r'[^0-9]', '', str(item))
    ThiessenLayer = gpd.read_file(filename=''.join(['ThBuf_', CuYear, '.shp']))
    Layer = gpd.read_file("WSFE_cumulativelayers.gdb", layer=item)
    print('Loaded original features and Thiessen layers. Joining together. %s\n' % time.ctime())
    ThiessenLayer = gpd.sjoin(ThiessenLayer, Layer, how='left', predicate='contains')
    print('Finished! Writing to file. %s\n' % time.ctime())
    ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThB2_', CuYear, '.shp']))
    del CuYear, ThiessenLayer, Layer
print('All years finished (except 2010).')

Thiessen polygons did not retain feature attributes. Need to join back on spatially.
Year: cu_sel_1999 Fri Oct 14 10:37:53 2022

Loaded original features and Thiessen layers. Joining together. Fri Oct 14 10:38:00 2022

Finished! Writing to file. Fri Oct 14 10:38:04 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1955055301.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThB2_', CuYear, '.shp']))


Year: cu_sel_2000 Fri Oct 14 10:38:12 2022

Loaded original features and Thiessen layers. Joining together. Fri Oct 14 10:38:18 2022

Finished! Writing to file. Fri Oct 14 10:38:22 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1955055301.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThB2_', CuYear, '.shp']))


Year: cu_sel_2001 Fri Oct 14 10:38:29 2022

Loaded original features and Thiessen layers. Joining together. Fri Oct 14 10:38:35 2022

Finished! Writing to file. Fri Oct 14 10:38:39 2022



...

Year: cu_sel_2014 Fri Oct 14 10:42:14 2022

Loaded original features and Thiessen layers. Joining together. Fri Oct 14 10:42:20 2022

Finished! Writing to file. Fri Oct 14 10:42:25 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1955055301.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThB2_', CuYear, '.shp']))


Year: cu_sel_2015 Fri Oct 14 10:42:33 2022

Loaded original features and Thiessen layers. Joining together. Fri Oct 14 10:42:39 2022

Finished! Writing to file. Fri Oct 14 10:42:44 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1955055301.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ThiessenLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThB2_', CuYear, '.shp']))


All years finished (except 2010).


In [ ]:
# for item in BuiltAreaList:
#     print('Loading Thiessen areas and buffered polygons from layer %s. %s\n' % (item, time.ctime()))
#     CuYear = re.sub(r'[^0-9]', '', item) # Pull the year of feature layer (e.g. "2005") from the numeric portion of the layer name.
#     ThiessenLayer = gpd.read_file(filename=''.join(['Thies_', CuYear, '.shp'])) # Read in the layer as a geodataframe.
#     BufferLayer = gpd.read_file(filename=''.join(['Buff_', CuYear, '.shp']))
#     print('Now clipping the Thiessen polygons with the buffer. %s\n' % time.ctime())
#     ThiessenBufferLayer = gpd.clip(ThiessenLayer, BufferLayer) # Clip the demarcated area so that coverage ends at the 2km mark. This will be both the mask used to reduce the file size of the population rasters, and the zones used to summarize the pop data during zonal statistics.
#     print('Clipped. Polygons did not retain feature attributes. Joining back on. %s\n' % time.ctime())
#     ThiessenBufferLayer = ThiessenBufferLayer.merge(Layer, how='left', left_index=True, right_index=True) # Voronoi function does not retain the attributes (leaves them all Null for whatever reason). Just joining it all back together.
#     print('Finished! Writing to file. %s\n' % time.ctime())
#     ThiessenBufferLayer.to_file(driver='ESRI Shapefile', filename=''.join(['ThBuf_', CuYear, '.shp']))
#     print(ThiessenBufferLayer.sample(5))
#     print('\nNext layer. %s\n' % time.ctime())
#     del CuYear, ThiessenLayer, BufferLayer, ThiessenBufferLayer

#### Zonal statistics
https://gis.stackexchange.com/questions/77993/issue-trying-to-create-zonal-statistics-using-gdal-and-python

In [105]:
def zonal_stats(feat, input_zone_polygon, input_value_raster, statistic='sum'):

    # Open data
    raster = gdal.Open(input_value_raster)
    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()

    # Get raster georeference info
    transform = raster.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]

    # Get extent of feat
    geom = feat.GetGeometryRef()
    if (geom.GetGeometryName() == 'MULTIPOLYGON'):
        count = 0
        pointsX = []; pointsY = []
        for polygon in geom:
            geomInner = geom.GetGeometryRef(count)    
            ring = geomInner.GetGeometryRef(0)
            numpoints = ring.GetPointCount()
            for p in range(numpoints):
                    lon, lat, z = ring.GetPoint(p)
                    pointsX.append(lon)
                    pointsY.append(lat)    
            count += 1
    elif (geom.GetGeometryName() == 'POLYGON'):
        ring = geom.GetGeometryRef(0)
        numpoints = ring.GetPointCount()
        pointsX = []; pointsY = []
        for p in range(numpoints):
                lon, lat, z = ring.GetPoint(p)
                pointsX.append(lon)
                pointsY.append(lat)

    else:
        sys.exit()

    xmin = min(pointsX)
    xmax = max(pointsX)
    ymin = min(pointsY)
    ymax = max(pointsY)

    # Specify offset and rows and columns to read
    xoff = int((xmin - xOrigin)/pixelWidth)
    yoff = int((yOrigin - ymax)/pixelWidth)
    xcount = int((xmax - xmin)/pixelWidth)+1
    ycount = int((ymax - ymin)/pixelWidth)+1

    # Create memory target raster
    target_ds = gdal.GetDriverByName('MEM').Create('', xcount, ycount, gdal.GDT_Byte)
    target_ds.SetGeoTransform((
        xmin, pixelWidth, 0,
        ymax, 0, pixelHeight,
    ))

    # Create for target raster the same projection as for the value raster
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(raster.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    # Rasterize zone polygon to raster
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values=[1])

    # Read raster as arrays
    banddataraster = raster.GetRasterBand(1)
    dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)

    bandmask = target_ds.GetRasterBand(1)
    datamask = bandmask.ReadAsArray(0, 0, xcount, ycount).astype(numpy.float)

    # Mask zone of raster
    zoneraster = numpy.ma.masked_array(dataraster,  numpy.logical_not(datamask))

    # Calculate statistics of zonal raster
    if (statistic == 'sum'):
        return numpy.sum(zoneraster)
    elif (statistic == 'mean'):
        return numpy.mean(zoneraster)
    elif (statistic == 'count'):
        return numpy.ma.count(zoneraster)
    elif (statistic == 'median'):
        return numpy.median(zoneraster)
    elif (statistic == 'min'):
        return numpy.min(zoneraster)
    elif (statistic == 'max'):
        return numpy.max(zoneraster)
    elif (statistic == 'std'):
        return numpy.std(zoneraster)
    else:
        return numpy.sum(zoneraster)
    
    
def loop_zonal_stats(input_zone_polygon, input_value_raster, statistic):
#         Parameters
#         ----------
#         input_zone_polygon: string
#             File path to read from using ogr.Open().
#         input_value_raster : string, default None
#             The OGR format driver used to write the vector file.
#             If not specified, it attempts to infer it from the file extension.
#             If no extension is specified, it saves ESRI Shapefile to a folder.
#         statistic : string, default = 'sum'
#             The statistic used to aggregate values in the input raster. 
#             Output will be a single value associated with the input zone.
#             Options: 'sum', 'mean', 'count', 'median', 'min', 'max', 'std'

    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()
    featList = range(lyr.GetFeatureCount())
    statDict = {}

    for FID in featList:
        feat = lyr.GetFeature(FID)
        statValue = zonal_stats(feat, input_zone_polygon, input_value_raster, statistic)
        statDict[FID] = statValue
    return statDict

In [113]:
PopRasterList = glob.glob(r'WorldPop_tifs_albers/*.tif') # List all files in the WorldPop folder that have a .tif extension
StatType = 'sum'

print('Running zonal statistics. \n')
for item in PopRasterList:
    Year = re.sub(r'[^0-9]', '', str(item)) # Get year of raster dataset. This will be used to choose the right polygon zones.
    print('Year: %s %s' % (Year, time.ctime()))
    RasterLayer = item
    ZoneLayerName = ''.join(['ThB2_', Year, '.shp'])
    ColumnName = ''.join(['Pop', Year]) # Zonal stats outputs will be named after the year and dataset, as a column in the settlements polygon.
    print('Zonal statistics (%s) for year %s. %s' % (StatType, Year, time.ctime()))
    zStats = loop_zonal_stats(ZoneLayerName, RasterLayer, StatType)
    print('Stats finished for year %s! Joining back onto Thiessen polygons. %s' % (Year, time.ctime()))
    zStats = pd.DataFrame.from_dict(zStats, orient = 'index', columns = [ColumnName])
    ZoneLayer = gpd.read_file(ZoneLayerName)
    ZoneWithStats = ZoneLayer.merge(zStats, how='left', left_index=True, right_index=True)
    print('And now joining the Thiessen polygon attributes back onto the settlements via spatial join. %s' % time.ctime())
    SettlementLayer = gpd.read_file("WSFE_cumulativelayers.gdb", layer=''.join(['cu_sel_', Year]))
    SettlementLayer = gpd.sjoin(SettlementLayer, ZoneWithStats, how='left', predicate='within')
    print('Joined. Writing the settlements to file with zonal info in attributes. %s\n' % time.ctime())
    SettlementLayer.to_file(''.join([StatType, '_', Year]))
    del Year, RasterLayer, ZoneLayer, ColumnName, zStats, ZoneWithStats, SettlementLayer
print('All years finished (except 2010). Outputs stored as shapefiles in project workspace.')

Running zonal statistics. 

Year: 2000 Sat Oct 15 14:02:13 2022
Zonal statistics (sum) for year 2000. Sat Oct 15 14:02:13 2022


C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:69: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)
C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamask = bandmas

Stats finished for year 2000! Joining back onto Thiessen polygons. Sat Oct 15 16:25:14 2022
And now joining the Thiessen polygon attributes back onto the settlements via spatial join. Sat Oct 15 16:25:15 2022
Joined. Writing the settlements to file with zonal info in attributes. Sat Oct 15 16:25:25 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\383848562.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SettlementLayer.to_file(''.join([StatType, '_', Year]))


Year: 2001 Sat Oct 15 16:25:59 2022
Zonal statistics (sum) for year 2001. Sat Oct 15 16:25:59 2022


...

C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:69: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)
C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamask = bandmas

Stats finished for year 2009! Joining back onto Thiessen polygons. Sun Oct 16 15:31:23 2022
And now joining the Thiessen polygon attributes back onto the settlements via spatial join. Sun Oct 16 15:31:25 2022
Joined. Writing the settlements to file with zonal info in attributes. Sun Oct 16 15:31:36 2022



C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\383848562.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SettlementLayer.to_file(''.join([StatType, '_', Year]))


Year: 2010 Sun Oct 16 15:32:08 2022
Zonal statistics (sum) for year 2010. Sun Oct 16 15:32:08 2022


AttributeError: 'NoneType' object has no attribute 'GetLayer'

### Scratch and notes

In [106]:
# Raster dataset
input_value_raster = r'WorldPop_tifs_albers/cmr_ppp_2000_UNadj.tif'
# Vector dataset(zones)
input_zone_polygon = 'ThB2_2000.shp'

Year2000 = loop_zonal_stats(input_zone_polygon, input_value_raster, statistic='sum')
Year2000

C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:69: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)
C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\1924271392.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamask = bandmas

{0: 830.1039101034403,
 1: 154.77438385039568,
 2: 2.4615999653525197e+41,
 3: 249.8179897069931,
 4: 186.9482328519225,
 5: 181.96457482874393,
 6: 1.3599999808577457e+39,
 7: 124.85239212960005,
 8: 86.9465765953064,
 9: 59.289542242884636,
 10: 36.25333206355572,
 11: 103.50703829526901,
 12: 203.06911008059978,
 13: 92.14826349914074,
 14: 1.1933999832026719e+41,
 15: 61.00516167283058,
 16: 59.620948418974876,
 17: 99.69450186938047,
 18: 86.21492018178105,
 19: 64.2365663908422,
 20: 105.92570769041777,
 21: 114.80186450481415,
 22: 123.16447875648737,
 23: 422.87874433398247,
 24: 1003.1307292878628,
 25: 3500.5727648735046,
 26: 6.867999903331616e+40,
 27: 1.6455999768378723e+41,
 28: 2.9375999586527307e+41,
 29: 2.400399966213921e+41,
 30: 2.733599961524069e+41,
 31: 8.251799883854372e+41,
 32: 8.251799883854372e+41,
 33: 1.1525999837769395e+41,
 34: 1.9379999727222876e+40,
 35: 1.6013999774599956e+41,
 36: 2.9069999590834314e+41,
 37: 2.658799962576893e+41,
 38: 4.78379993266

In [107]:
poly = gpd.read_file('ThB2_2000.shp')
zstats = pd.DataFrame.from_dict(Year2000, orient = 'index', columns = ['Pop2000'])
poly = poly.merge(zstats, how='left', left_index=True, right_index=True)
poly.to_file(driver='ESRI Shapefile', filename='test2000_zonal.shp')

##### EARLIER VERSION CODE BLOCKS

In [100]:
daShapefile = r'ThB2_2000.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(daShapefile, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    print('Could not open %s' % (daShapefile))
else:
    print('Opened %s' % (daShapefile))
    layer = dataSource.GetLayer()
    featureCount = layer.GetFeatureCount()
    print("Number of features in %s: %d" % (os.path.basename(daShapefile),featureCount))

Opened ThB2_2000.shp
Number of features in ThB2_2000.shp: 13422


##### VALIDATION

In [34]:
layer2010 = gpd.read_file("WSFE_cumulativelayers.gdb", layer='cu_sel_2010')

ext_vertices = []
int_vertices = []

for i, row in layer2010.iterrows():
    # It's better to check if multigeometry
    multi = row.geometry.type.startswith("Multi")

    if multi:
        n = 0
        allparts = []
        # iterate over all parts of multigeometry
        for part in row.geometry:
            part_length = len(part.exterior.coords)
            allparts.append(part_length)
            n += min(allparts)
    else:
        n = len(row.geometry.exterior.coords)
    ext_vertices.append(n) ###


layer2010["ext_vertices"] = ext_vertices
print(layer2010.sample(10))

invalid2010 = layer2010[layer2010['ext_vertices']<4]
print(invalid2010)

C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\3910002172.py:14: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for part in row.geometry:


       GRID3_splitID  MAX_year  Shape_Length     Shape_Area  \
1372          5075.0      2001      112.9885     797.900071   
11618       194201.0      2010    13163.1764  710132.680130   
939           2915.0      1985      508.4489    9574.813560   
12328       196431.0      2010     1920.8071   34309.784960   
9330        190982.0      2005      395.4604    3191.611582   
9062        190637.0      2008     3163.6815   79790.197745   
1287          4569.0      1994      112.9888     797.904308   
1606          6866.0      2010     1299.3698   43884.618293   
9546        191291.0      2010     8926.1007  539381.711896   
11627       194217.0      2010     3954.6030  136441.222819   

                                                geometry  ext_vertices  
1372   MULTIPOLYGON (((-1663060.818 480910.238, -1663...             5  
11618  MULTIPOLYGON (((-1415474.477 582543.513, -1415...            79  
939    MULTIPOLYGON (((-1413666.659 325409.633, -1413...            13  
12328  MULTIPO

In [36]:
layer2011 = gpd.read_file("WSFE_cumulativelayers.gdb", layer='cu_sel_2011')

ext_vertices = []
int_vertices = []

for i, row in layer2011.iterrows():
    # It's better to check if multigeometry
    multi = row.geometry.type.startswith("Multi")

    if multi:
        n = 0
        allparts = []
        # iterate over all parts of multigeometry
        for part in row.geometry:
            part_length = len(part.exterior.coords)
            allparts.append(part_length)
            n += min(allparts)
    else:
        n = len(row.geometry.exterior.coords)
    ext_vertices.append(n) ###


layer2011["ext_vertices"] = ext_vertices
print(layer2011.sample(10))

invalid2011 = layer2011[layer2011['ext_vertices']<4]
print(invalid2011)

C:\Users\wb527163\AppData\Local\Temp\ipykernel_11452\3860069944.py:14: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for part in row.geometry:


       GRID3_splitID  MAX_year  Shape_Length    Shape_Area  \
7055         73057.0      2011      903.9094  2.313916e+04   
3958         24195.0      2011      564.9430  1.436221e+04   
13552       205431.0      2005     1355.8640  3.191609e+04   
11700       194166.0      2011    10620.9316  6.598650e+05   
12265       195920.0      2011      112.9886  7.979015e+02   
7575        187383.0      1985     1977.3014  8.537550e+04   
58              59.0      2011    30337.4462  4.875979e+06   
12646       197415.0      1985      282.4714  4.787406e+03   
2502         13019.0      1985      225.9772  3.191606e+03   
4263         27362.0      2008     1186.3810  2.872449e+04   

                                                geometry  ext_vertices  
7055   MULTIPOLYGON (((-1134132.782 659742.996, -1134...            15  
3958   MULTIPOLYGON (((-1366296.174 441703.183, -1366...            14  
13552  MULTIPOLYGON (((-1101733.292 1460069.727, -110...            31  
11700  MULTIPOLYGON (((-1

In [39]:
print(min(layer2010['ext_vertices']))
print(min(layer2011['ext_vertices']))
print(layer2010['ext_vertices'].isna().sum())
print(layer2011['ext_vertices'].isna().sum())

5
5
0
0


In [50]:
non_multi = []

for i, row in layer2010.iterrows():
    multi = row.geometry.type.startswith("Multi")
    
    if multi:
        pass
    else:
        n = row
    non_multi.append(n) ###
print(len(non_multi))

13723


In [41]:
ext_vertices = []
int_vertices = []

for i, row in layer2010.iterrows():
    # It's better to check if multigeometry
    try:
        n = len(row.geometry.interior.coords)
    except:
        pass
    int_vertices.append(n) ###

layer2010["int_vertices"] = int_vertices
print(layer2010.sample(10))

invalid2010 = layer2010[layer2010['int_vertices']<4]
print(invalid2010)

       GRID3_splitID  MAX_year  Shape_Length     Shape_Area  \
13142       203768.0      2010     2485.7500   73406.985880   
6648         63868.0      2005      621.4370   15958.019776   
2278         11588.0      2010      338.9658    3989.510240   
12022       195442.0      2009     2994.1983   94152.397607   
11748       194505.0      2010     1186.3804   23937.055793   
5135         41701.0      2002      225.9775    2393.711511   
10451       192657.0      2010     1581.8410   45480.412785   
7985        189282.0      2010    19264.5613  797104.035005   
13392       205224.0      2005      903.9094   25532.868645   
3956         24537.0      1985      564.9432    9574.827684   

                                                geometry  ext_vertices  \
13142  MULTIPOLYGON (((-1037499.254 1254119.697, -103...            62   
6648   MULTIPOLYGON (((-1493606.116 669883.726, -1493...            11   
2278   MULTIPOLYGON (((-1421067.414 413597.260, -1421...            12   
12022  MUL